## =======================处理变长序列============================

In [1]:
import torch
import torch.nn as nn
import numpy as np


In [2]:
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence

In [4]:
>>> a = torch.ones(2, 5)
>>> b = torch.ones(3, 5)
>>> c = torch.ones(4, 5)

In [5]:
pad_sequence([a, b, c])

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1.]]])

In [7]:
batch = [a, b, c]

In [17]:
batch_len = np.array([2, 3, 4])

In [10]:
paded_batch = pad_sequence(batch)

In [23]:
sort_idx = np.argsort(-batch_len)
sort_idx

array([2, 1, 0], dtype=int64)

In [24]:
torch.LongTensor(sort_idx)

tensor([2, 1, 0])

In [25]:
paded_batch

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [1., 1., 1., 1., 1.]]])

In [30]:
sort_padded = paded_batch[:, sort_idx, :]
sort_padded

tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]],

        [[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0.]],

        [[1., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]])

In [32]:
pack_padded = pack_padded_sequence(sort_padded, [4, 3, 2])
pack_padded

PackedSequence(data=tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]]), batch_sizes=tensor([3, 3, 2, 1]))

In [36]:
hidden = (torch.zeros((1, 3, 10), dtype=torch.float), torch.zeros((1, 3, 10), dtype=torch.float))
hidden

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]),
 tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]))

In [37]:
lstm = nn.LSTM(5, 10)

In [38]:
lstm_out, hidden = lstm(pack_padded, hidden)

In [41]:
lstm_out.size()

AttributeError: 'PackedSequence' object has no attribute 'size'

In [43]:
hidden[0].size()

torch.Size([1, 3, 10])

In [44]:
hidden[1].size()

torch.Size([1, 3, 10])

In [47]:
hidden[0]

tensor([[[ 0.3000,  0.0716,  0.0656,  0.4211,  0.2340, -0.0994, -0.0338,
          -0.2359, -0.1413, -0.0651],
         [ 0.2641,  0.0624,  0.0674,  0.3852,  0.1975, -0.0885, -0.0325,
          -0.2255, -0.1321, -0.0613],
         [ 0.2079,  0.0485,  0.0681,  0.3215,  0.1478, -0.0705, -0.0268,
          -0.2014, -0.1151, -0.0544]]], grad_fn=<ViewBackward>)

In [48]:
hidden[1]

tensor([[[ 0.7395,  0.1979,  0.1487,  0.8126,  0.4700, -0.1719, -0.1030,
          -0.6727, -0.3121, -0.1590],
         [ 0.6420,  0.1731,  0.1513,  0.7235,  0.3887, -0.1526, -0.0992,
          -0.6232, -0.2903, -0.1481],
         [ 0.4997,  0.1356,  0.1504,  0.5831,  0.2854, -0.1208, -0.0820,
          -0.5284, -0.2509, -0.1291]]], grad_fn=<ViewBackward>)

In [70]:
pad_packed = pad_packed_sequence(lstm_out)
pad_packed

(tensor([[[ 0.1221,  0.0279,  0.0575,  0.2057,  0.0816, -0.0408, -0.0143,
           -0.1426, -0.0811, -0.0400],
          [ 0.1221,  0.0279,  0.0575,  0.2057,  0.0816, -0.0408, -0.0143,
           -0.1426, -0.0811, -0.0400],
          [ 0.1221,  0.0279,  0.0575,  0.2057,  0.0816, -0.0408, -0.0143,
           -0.1426, -0.0811, -0.0400]],
 
         [[ 0.2079,  0.0485,  0.0681,  0.3215,  0.1478, -0.0705, -0.0268,
           -0.2014, -0.1151, -0.0544],
          [ 0.2079,  0.0485,  0.0681,  0.3215,  0.1478, -0.0705, -0.0268,
           -0.2014, -0.1151, -0.0544],
          [ 0.2079,  0.0485,  0.0681,  0.3215,  0.1478, -0.0705, -0.0268,
           -0.2014, -0.1151, -0.0544]],
 
         [[ 0.2641,  0.0624,  0.0674,  0.3852,  0.1975, -0.0885, -0.0325,
           -0.2255, -0.1321, -0.0613],
          [ 0.2641,  0.0624,  0.0674,  0.3852,  0.1975, -0.0885, -0.0325,
           -0.2255, -0.1321, -0.0613],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
            0.00

In [69]:
unsort_idx = np.argsort(sort_idx)
unsort_idx

array([2, 1, 0], dtype=int64)

In [75]:
pad_packed[0][:, unsort_idx, :]

tensor([[[ 0.1221,  0.0279,  0.0575,  0.2057,  0.0816, -0.0408, -0.0143,
          -0.1426, -0.0811, -0.0400],
         [ 0.1221,  0.0279,  0.0575,  0.2057,  0.0816, -0.0408, -0.0143,
          -0.1426, -0.0811, -0.0400],
         [ 0.1221,  0.0279,  0.0575,  0.2057,  0.0816, -0.0408, -0.0143,
          -0.1426, -0.0811, -0.0400]],

        [[ 0.2079,  0.0485,  0.0681,  0.3215,  0.1478, -0.0705, -0.0268,
          -0.2014, -0.1151, -0.0544],
         [ 0.2079,  0.0485,  0.0681,  0.3215,  0.1478, -0.0705, -0.0268,
          -0.2014, -0.1151, -0.0544],
         [ 0.2079,  0.0485,  0.0681,  0.3215,  0.1478, -0.0705, -0.0268,
          -0.2014, -0.1151, -0.0544]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000],
         [ 0.2641,  0.0624,  0.0674,  0.3852,  0.1975, -0.0885, -0.0325,
          -0.2255, -0.1321, -0.0613],
         [ 0.2641,  0.0624,  0.0674,  0.3852,  0.1975, -0.0885, -0.0325,
          -0.2255, -0.1321, -0.0613

In [77]:
hidden[0][-1][unsort_idx]

tensor([[ 0.2079,  0.0485,  0.0681,  0.3215,  0.1478, -0.0705, -0.0268, -0.2014,
         -0.1151, -0.0544],
        [ 0.2641,  0.0624,  0.0674,  0.3852,  0.1975, -0.0885, -0.0325, -0.2255,
         -0.1321, -0.0613],
        [ 0.3000,  0.0716,  0.0656,  0.4211,  0.2340, -0.0994, -0.0338, -0.2359,
         -0.1413, -0.0651]], grad_fn=<TakeBackward>)

In [60]:
d = np.array([5,8,2,9,4])

In [61]:
a=np.argsort(-d)
a

array([3, 1, 0, 4, 2], dtype=int64)

In [62]:
d[a]

array([9, 8, 5, 4, 2])

In [64]:
a_ = np.argsort(a)
a_

array([2, 1, 4, 0, 3], dtype=int64)

In [65]:
d[a][a_]

array([5, 8, 2, 9, 4])

### 总结： batch seq -> sort -> pad and pack ->process using RNN -> unpack ->unsort

## ==========================================================================

## =============================cnn=================================

Our inputs are a batch of 2D text data. Let's make an input with 64 samples, where each sample has 8 words and each word is represented by a array of 10 values (one hot encoded with vocab size of 10). This gives our inputs the size (64, 8, 10). The PyTorch CNN modules prefer inputs to have the channel dim (one hot vector dim in our case) to be in the second position, so our inputs are of shape (64, 10, 8).

<img src="https://raw.githubusercontent.com/GokuMohandas/practicalAI/master/images/cnn_text1.png" width=400>



We want to convolve on this input using filters. For simplicity we will use just 5 filters that is of size (1, 2) and has the same depth as the number of channels (one_hot_size). This gives our filter a shape of (5, 2, 10) but recall that PyTorch CNN modules prefer to have the channel dim (one hot vector dim in our case) to be in the second position so the filter is of shape (5, 10, 2).

<img src="https://raw.githubusercontent.com/GokuMohandas/practicalAI/master/images/cnn_text2.png" width=400>


When we apply this filter on our inputs, we receive an output of shape (64, 5, 7). We get 64 for the batch size, 5 for the channel dim because we used 5 filters and 7 for the conv outputs because:

$\frac{W - F + 2P}{S} + 1 = \frac{8 - 2 + 2(0)}{1} + 1 = 7$

where:

W: width of each input
F: filter size
P: padding
S: stride
<img src="https://raw.githubusercontent.com/GokuMohandas/practicalAI/master/images/cnn_text3.png" width=400>

#### torch.Conv1d--API

https://pytorch.org/docs/stable/nn.html?highlight=conv#torch.nn.Conv1d

In the simplest case, the output value of the layer with input size(N, Cin, L) and output(N, Cout, Lout) can be precisely described as:

    out(Ni, Coutj) = bias(Coutj) + Σweight(Coutj, k) ⋆ input(Ni, k); k∈[0, Cin-1]
    where ⋆ is the valid cross-correlation operator, N is a batch size, C denotes a number of channels, L is a length of signal sequence.
    

#### 好好理解上面的公式，这种卷积方式是等同于之前所认识的卷积，不过卷积的次序有些不一样（将词向量理解为通道，先在每个通道上进行句长的卷积）

Lout = ⌊(Lin + 2 * padding - dilation * (kernel_size-1) -1) / stride +1⌋

注意点：
1.input shape(N, Cin, Lin); output shape(N, Cout, Lout); weight shape(out_channels, in_channels, kernel_size); bias shape(out_channels)

2.dilation (int or tuple, optional) – Spacing between kernel elements. Default: 1
( controls the spacing between the kernel points; also known as the à trous algorithm. It is harder to describe, but this link has a nice visualization of what dilation does.)
(there are usually d=1 spaces inserted between kernel elements such that d = 1 corresponds to a regular convolution.)

3.groups (int, optional) – Number of blocked connections from input channels to output channels. Default: 1

    (controls the connections between inputs and outputs. in_channels and out_channels must both be divisible by groups. For example,
    At groups=1, all inputs are convolved to all outputs.
    At groups=2, the operation becomes equivalent to having two conv layers side by side, each seeing half the input channels, and producing half the output channels, and both subsequently concatenated.
    At groups= in_channels, each input channel is convolved with its own set of filters, of size ⌊Cout/Cin⌋)

## ===================normalization=========

本文归纳了目前深度学习技术中针对神经元进行Normalization操作的若干种模型，可以看出，所有模型都采取了类似的步骤和过程，将神经元的激活值重整为均值为0方差为1的新数值，最大的不同在于计算统计量的神经元集合S的划分方法上。BN采用了同一个神经元，但是来自于Mini-Batch中不同训练实例导致的不同激活作为统计范围。而为了克服Mini-Batch带来的弊端，后续改进方法抛弃了Mini-Batch的思路，只用当前训练实例引发的激活来划分集合S的统计范围，概括而言，LayerNorm采用同隐层的所有神经元；InstanceNorm采用CNN中卷积层的单个通道作为统计范围，而GroupNorm则折衷两者，采用卷积层的通道分组，在划分为同一个分组的通道内来作为通道范围。

至于各种Normalization的适用场景，可以简洁归纳如下：对于RNN的神经网络结构来说，目前只有LayerNorm是相对有效的；如果是GAN等图片生成或图片内容改写类型的任务，可以优先尝试InstanceNorm；如果使用场景约束BatchSize必须设置很小，无疑此时考虑使用GroupNorm；而其它任务情形应该优先考虑使用BatchNorm。